In [ ]:
import toml
import re
import urllib
import benchlingapi
import requests_html

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api

# Code

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

# Reference

In [ ]:
folder = api.get_folder(session, "Test2", project="Test")

In [ ]:
dna = session.DNASequence.find_by_name("V37m", folder_id=folder.id)

In [ ]:
url = "https://www.addgene.org/108254/"
# url = 'https://www.addgene.org/108537/'
# url = 'http://www.addgene.org/121010/'
api.get_addgene_info(url)

In [ ]:
# url = 'https://www.addgene.org/kits/marionette-sensor-collection/#kit-contents'
url = "http://www.addgene.org/kits/murray-cidar-moclo-v1/"
kit = api.get_addgene_kit(url, include_info=False, include_sequences=False)

In [ ]:
# gb = get_genbank(addgene_sequences('http://www.addgene.org/121010/')['depositor_full'][0])[0]
gb = get_genbank(
    addgene_sequences("https://www.addgene.org/108537/")["addgene_full"][0]
)[0]

In [ ]:
dna = genbank_to_benchling(session, gb, "seq3", folder.id, accession="addgene_blah")
dna.save()